Make ROI Spheres from Coordinates
===

In [1]:
from nilearn import datasets, image, plotting
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from numpy.linalg import inv
import os
from tqdm import tqdm
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)
/usr/local/lib/python2.7/dist-packages/nilearn/__init__.py:73: DeprecationWarning: Python2 support is deprecated and will be removed in the next release. Consider switching to Python 3.6 or 3.7.
  _python_deprecation_warnings()


By default, this notebook assumes that the coordinates list is in MNI space. If it is already in voxel space, 
add `voxel_coord = True` to the call to `make_spheres` in the last cell 

In [2]:
mni_coords = pd.read_csv('./tutorial_coords.csv', header = None)

In [3]:
mni_coords = mni_coords.dropna()

In [4]:
mni_brain = datasets.load_mni152_brain_mask()

In [5]:
brain_coords_list = []
for i in range(0, mni_brain.shape[0]):
    for j in range(0, mni_brain.shape[1]):
        for k in range(0, mni_brain.shape[2]):
            brain_coords_list.append([i, j, k])

In [6]:
# Specify the radius of the sphere (in voxels) here. For the default mni template brain, each voxel is 2mm
nbrs = NearestNeighbors(metric='euclidean', radius = 1).fit(brain_coords_list)

In [7]:
def make_sphere(nbrs, coord, coord_space, brain_mask, voxel_coord = False):
    #Transform to voxelwise coords
    if voxel_coord == False:
        inv_affine = inv(mni_brain.affine)
        coord = image.coord_transform(coord[0], coord[1], coord[2], inv_affine)
        
    indices = nbrs.radius_neighbors([coord])[1]
    blank = np.zeros(brain_mask.shape)
    #print(coord_space[indices[0]])
    for c in coord_space[indices[0]]:
        blank[c[0], c[1], c[2]] = 1
    sphere_img = image.new_img_like(brain_mask, blank)
    return sphere_img
    


In [8]:
# Specify output directory

outdir = './data/spheres'
os.mkdir(outdir)

for c in tqdm(mni_coords.iterrows()):
    sphere = make_sphere(nbrs, c[1], np.asarray(brain_coords_list), mni_brain)
    sphere.to_filename(outdir + '/' + str(c[1][0])+ '_' + str(c[1][1]) + '_' + str(c[1][2]) + '.nii.gz')

10it [00:06,  1.56it/s]
